In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.insert(1, '/home/federico/Università/Tesi_magistrale/my_programs/')

from contact_matrix_utils import *
from scipy.stats import pearsonr, spearmanr
from scipy.spatial.distance import squareform
from src.utilities import *

Questo notebook serve per calcolare la media delle pi, coseg e NPMI per ogni coppia di cromosomi (esclusi quelli sessuali) sui dataset mesc_46C_1Mb_481x1

In [2]:
save_folder = "/home/federico/Università/Tesi_magistrale/images/13_averages_genomewide_1Mb.ipynb_output/"

# Import coseg #

In [3]:
name_root_1NP_1Mb = "mesc_46C_1Mb_481x1"

path_1 =      r"/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/data/" + name_root_1NP_1Mb + "/"
file_name_1 = "segregation_" + name_root_1NP_1Mb +  ".pkl"
with open(path_1 + file_name_1, 'rb') as f:
    segregation_table_1NP_1Mb = pickle.load(f)

start_chr, stop_chr = 1, 19
stop_chr += 1
chromosomes = []
n_chromosomes = stop_chr - start_chr

l = []
size = 0

for i in range(start_chr, stop_chr):
    chrA = 'chr' + str(i)
    chromosomes += [chrA]
    s = segregation_table_1NP_1Mb[chrA]['segregation_table'].shape[0]
    size += s
    l.append(s) 

chr_windows_sizes = np.array(l)

# Average $\pi$ #

## mESC 1NP 1Mb ##

In [4]:
genomewide_1NP_1Mb_average_pi = np.zeros((19, 19))*np.nan
genomewide_1NP_1Mb_average_sign_pi = np.zeros((19, 19))*np.nan

for i in range(n_chromosomes):
    for j in range(i, n_chromosomes):
        start_row    = np.sum(chr_windows_sizes[:i])
        stop_row     = start_row + chr_windows_sizes[i]
        start_column = np.sum(chr_windows_sizes[:j])
        stop_column  = start_column + chr_windows_sizes[j]
        chrA = chromosomes[i]
        chrB = chromosomes[j]
        if(chrA!=chrB):

            pi = np.load(data_path + name_root_1NP_1Mb + "/PI2_inter_beta_evaluation/" + "PI2_inter_" +  chrA + "_" + chrB + "_" + name_root_1NP_1Mb + ".npy")
            sign_pi = np.load(data_path + name_root_1NP_1Mb + "/PI2_inter_beta_evaluation/" + "PI2_inter_significant_95_" +  chrA + "_" + chrB + "_" + name_root_1NP_1Mb + ".npy")



            genomewide_1NP_1Mb_average_pi[i,j] = np.nanmean(pi)
            genomewide_1NP_1Mb_average_sign_pi[i,j] = np.nanmean(sign_pi)

            genomewide_1NP_1Mb_average_pi[j,i] = genomewide_1NP_1Mb_average_pi[i,j]
            genomewide_1NP_1Mb_average_sign_pi[j,i] = genomewide_1NP_1Mb_average_sign_pi[i,j]

            del pi, sign_pi
        

        else:
            pi = squareform(np.loadtxt(data_path + name_root_1NP_1Mb + "/PI2/" + "PI2_" + chrA + "_" + name_root_1NP_1Mb + ".txt") )
            sign_pi = squareform(np.loadtxt(data_path + name_root_1NP_1Mb + "/PI2/" + "PI2_significant_95_" + chrA + "_" + name_root_1NP_1Mb + ".txt") )

            np.fill_diagonal(pi, np.nan)
            np.fill_diagonal(sign_pi, np.nan)
            
            genomewide_1NP_1Mb_average_pi[i,j] = np.nanmean(pi)
            genomewide_1NP_1Mb_average_sign_pi[i,j] = np.nanmean(sign_pi)

In [14]:
temp_1 = np.copy(genomewide_1NP_1Mb_average_pi)
temp_2 = np.copy(genomewide_1NP_1Mb_average_sign_pi)

np.fill_diagonal(temp_1, np.nan)
np.fill_diagonal(temp_2, np.nan)

fig1, _, _ = plot_genomewide_averages(genomewide_1NP_1Mb_average_pi, vmin = np.nanpercentile(temp_1, 3), vmax = np.nanpercentile(temp_1, 97), title = "Average $\pi$ - " + name_root_1NP_1Mb)
fig2, _, _ = plot_genomewide_averages(genomewide_1NP_1Mb_average_sign_pi, vmin = np.nanpercentile(temp_2, 3), vmax = np.nanpercentile(temp_2, 97), title = "Average significant $\pi$ 95% - " + name_root_1NP_1Mb)

fig1.savefig(save_folder + 'beagrie_mesc_1NP/beagrie_mesc_1NP_1Mb_average_pi.svg' , format = 'svg', dpi = 300)
fig2.savefig(save_folder + 'beagrie_mesc_1NP/beagrie_mesc_1NP_1Mb_average_sign_pi.svg' , format = 'svg', dpi = 300)

plt.close(fig1)
plt.close(fig2)

del fig1, fig2, temp_1, temp_2

In [6]:
t =  genomewide_1NP_1Mb_average_pi
t_nodiag = np.copy(t);  np.fill_diagonal(t_nodiag, np.nan)

print(np.nanmean(np.diag(t))*100, np.nanstd(np.diag(t))*100, np.nanmean(t_nodiag)*100, np.nanstd(t_nodiag)*100 )

del t, t_nodiag

t =  genomewide_1NP_1Mb_average_sign_pi
t_nodiag = np.copy(t);  np.fill_diagonal(t_nodiag, np.nan)

print(np.nanmean(np.diag(t))*100, np.nanstd(np.diag(t))*100, np.nanmean(t_nodiag)*100, np.nanstd(t_nodiag)*100 )

del t, t_nodiag

3.4847191409755194 0.1910403509408472 1.5083059501927243 0.07851594133549211
7.9949397780294476 0.1966990001849021 3.4484079650201274 0.10556871735954686


In [7]:
3.48/4.78, 1.51/1.83

(0.7280334728033473, 0.825136612021858)

# Average cosegregation #

## mESC 1NP 1Mb ##

In [8]:
genomewide_1NP_1Mb_average_coseg = np.zeros((19, 19))*np.nan

for i in range(n_chromosomes):
    for j in range(i, n_chromosomes):
        start_row    = np.sum(chr_windows_sizes[:i])
        stop_row     = start_row + chr_windows_sizes[i]
        start_column = np.sum(chr_windows_sizes[:j])
        stop_column  = start_column + chr_windows_sizes[j]
        chrA = chromosomes[i]
        chrB = chromosomes[j]
        if(chrA!=chrB):

            coseg = compute_tube_cosegregation_matrix_offdiag(segregation_table_1NP_1Mb[chrA]['segregation_table'], segregation_table_1NP_1Mb[chrB]['segregation_table'])

            F_arr_A, F_arr_B = compute_tube_segregation_frequency_inter(segregation_table_1NP_1Mb[chrA]['segregation_table'], segregation_table_1NP_1Mb[chrB]['segregation_table'])

            coseg[:, np.isnan(F_arr_B)] = np.nan
            coseg[np.isnan(F_arr_A), :] = np.nan

            genomewide_1NP_1Mb_average_coseg[i,j] = np.nanmean(coseg)
            genomewide_1NP_1Mb_average_coseg[j,i] = genomewide_1NP_1Mb_average_coseg[i,j] 

        else:
            coseg = compute_tube_cosegregation_matrix(segregation_table_1NP_1Mb[chrA]['segregation_table'])
            np.fill_diagonal(coseg, np.nan)

            F_arr_A = compute_tube_segregation_frequency(segregation_table_1NP_1Mb[chrA]['segregation_table'])

            coseg[:, np.isnan(F_arr_A)] = np.nan
            coseg[np.isnan(F_arr_A), :] = np.nan

            genomewide_1NP_1Mb_average_coseg[i,j] = np.nanmean(coseg)



In [13]:
temp_1 = np.copy(genomewide_1NP_1Mb_average_coseg)

np.fill_diagonal(temp_1, np.nan)

fig, _, _ = plot_genomewide_averages(genomewide_1NP_1Mb_average_coseg, vmin = np.nanpercentile(genomewide_1NP_1Mb_average_coseg, 3), vmax = np.nanpercentile(genomewide_1NP_1Mb_average_coseg, 97), title = "Average cosegregation - " + name_root_1NP_1Mb)

fig.savefig(save_folder + 'beagrie_mesc_1NP/beagrie_mesc_1NP_1Mb_average_coseg.svg' , format = 'svg', dpi = 300)

plt.close(fig)

del fig, temp_1

# Average NPMI #

## mESC 1NP 1Mb ##

In [10]:
genomewide_1NP_1Mb_average_npmi = np.zeros((19, 19))*np.nan

for i in range(n_chromosomes):
    for j in range(i, n_chromosomes):
        start_row    = np.sum(chr_windows_sizes[:i])
        stop_row     = start_row + chr_windows_sizes[i]
        start_column = np.sum(chr_windows_sizes[:j])
        stop_column  = start_column + chr_windows_sizes[j]
        chrA = chromosomes[i]
        chrB = chromosomes[j]
        if(chrA!=chrB):

            npmi = compute_npmi_inter(segregation_table_1NP_1Mb[chrA]['segregation_table'], segregation_table_1NP_1Mb[chrB]['segregation_table'])
            genomewide_1NP_1Mb_average_npmi[i,j] = np.nanmean(npmi)
            genomewide_1NP_1Mb_average_npmi[j,i] = genomewide_1NP_1Mb_average_npmi[i,j] 

        else:
            npmi = compute_npmi(segregation_table_1NP_1Mb[chrA]['segregation_table'])
            np.fill_diagonal(npmi, np.nan)
            genomewide_1NP_1Mb_average_npmi[i,j] = np.nanmean(npmi)

/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/src/utilities.py:40: RuntimeWarning: divide by zero encountered in log
  npmi_mat = - np.log(F_ij / (F_i * F_j)) / np.log(F_ij)
/home/federico/Università/Tesi_magistrale/SLICE/slice_repository/src/utilities.py:56: RuntimeWarning: divide by zero encountered in log
  npmi_mat = - np.log(F_ij / (F_i * F_j)) / np.log(F_ij)


In [11]:
temp_1 = np.copy(genomewide_1NP_1Mb_average_npmi)

np.fill_diagonal(temp_1, np.nan)

fig, _,  _  = plot_genomewide_averages(genomewide_1NP_1Mb_average_npmi, vmin = np.nanpercentile(temp_1, 3), vmax = np.nanpercentile(temp_1, 97), title="Average NPMI - " + name_root_1NP_1Mb)

fig.savefig(save_folder + 'beagrie_mesc_1NP/beagrie_mesc_1NP_1Mb_average_NPMI.svg' , format = 'svg', dpi = 300)

plt.close(fig)

del fig, temp_1